<a href="https://colab.research.google.com/github/epicskills1/Alemeno-Assignment/blob/main/ContentEngineBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extract Text from pdfs

In [1]:
# Install PyMuPDF
!pip install PyMuPDF

import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Upload files
from google.colab import files
uploaded = files.upload()

# Extract text from each PDF
alphabet_text = extract_text_from_pdf("Alphabet_10K.pdf")
tesla_text = extract_text_from_pdf("Tesla_10K.pdf")
uber_text = extract_text_from_pdf("Uber_10K.pdf")

# Save extracted text to variables or files
alphabet_text, tesla_text, uber_text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 24.7 MB/s eta 0:00:00


Saving Uber_10K.pdf to Uber_10K.pdf
Saving Tesla_10K.pdf to Tesla_10K.pdf
Saving Alphabet_10K.pdf to Alphabet_10K.pdf


('UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________\nFORM 10-K \n___________________________________________\n(Mark One)\n☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2023 \nOR\n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from              to             .\nCommission file number: 001-37580 \n___________________________________________\nAlphabet Inc. \n(Exact name of registrant as specified in its charter)\n___________________________________________\nDelaware\n61-1767919\n(State or other jurisdiction of incorporation or organization)\n(I.R.S. Employer Identification No.)\n1600 Amphitheatre Parkway \nMountain View, CA 94043 \n(Address of principal executive offices, including zip code)\n(650) 253-0000 \n(Registrant\'s telephone number, including area code)\nSecur

Generate Embeddings

In [3]:
# Install SentenceTransformers
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

alphabet_embeddings = model.encode(alphabet_text.split('\n'))
tesla_embeddings = model.encode(tesla_text.split('\n'))
uber_embeddings = model.encode(uber_text.split('\n'))

# Save embeddings to variables
alphabet_embeddings, tesla_embeddings, uber_embeddings


(array([[ 0.06430631, -0.00233027,  0.00298114, ..., -0.00998688,
          0.00754462,  0.02378991],
        [-0.03600856, -0.00762786, -0.08394636, ..., -0.04425151,
          0.00894299,  0.01707131],
        [-0.0490362 ,  0.02523387,  0.03324314, ...,  0.03459061,
         -0.04973461,  0.00496775],
        ...,
        [-0.06946586, -0.05853913, -0.00857873, ..., -0.05465923,
          0.06013927, -0.01246304],
        [ 0.04487905,  0.08137882, -0.06480395, ...,  0.01643461,
         -0.01270881, -0.03251163],
        [-0.11883841,  0.04829865, -0.00254803, ...,  0.12640952,
          0.04654907, -0.01571731]], dtype=float32),
 array([[ 0.0643063 , -0.00233029,  0.00298116, ..., -0.00998686,
          0.00754464,  0.02378989],
        [-0.03600856, -0.00762786, -0.08394636, ..., -0.04425151,
          0.00894299,  0.01707131],
        [-0.0490362 ,  0.02523387,  0.03324314, ...,  0.03459061,
         -0.04973461,  0.00496775],
        ...,
        [-0.05527474,  0.11858369, -0.1

Store in Vector Store using FAISS

In [4]:
# Install FAISS
!pip install faiss-cpu

import faiss
import numpy as np

def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index

alphabet_index = create_faiss_index(alphabet_embeddings)
tesla_index = create_faiss_index(tesla_embeddings)
uber_index = create_faiss_index(uber_embeddings)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 16.2 MB/s eta 0:00:00


Query the FAISS Index

In [5]:
def query_index(index, query_embedding, texts):
    D, I = index.search(query_embedding, k=5)
    return [texts[i] for i in I[0]]

def get_query_embedding(query):
    return model.encode([query])

# Example query
query = "What are the risk factors associated with Google and Tesla?"
query_embedding = get_query_embedding(query)

alphabet_results = query_index(alphabet_index, query_embedding, alphabet_text.split('\n'))
tesla_results = query_index(tesla_index, query_embedding, tesla_text.split('\n'))
uber_results = query_index(uber_index, query_embedding, uber_text.split('\n'))

print("Alphabet Inc. Results:")
for result in alphabet_results:
    print(result)

print("\nTesla, Inc. Results:")
for result in tesla_results:
    print(result)

print("\nUber Technologies, Inc. Results:")
for result in uber_results:
    print(result)


Alphabet Inc. Results:
spending. Additionally, the overall increase in Google subscriptions, platforms, and devices revenues was adversely 
Risks Related to our Industry 
sustainability and climate change since Google’s founding more than 25 years ago. 
Risk Factors
RISK FACTORS

Tesla, Inc. Results:
(e.g., vehicle functionality, Autopilot and FSD Capability), compliance, finance, data privacy, and other matters related to Tesla’s business, its personnel,
particular, Tesla’s products, business, results of operations, and statements and actions of Tesla and its management are subject to significant amounts of
Risk Factors
and personal information. Tesla has made notifications to potentially affected individuals (current and former employees) and regulatory authorities and
business, financial condition and future results. The risks described below are not the only risks facing our company. Risks and uncertainties not currently

Uber Technologies, Inc. Results:
Risk Factors
company, busin

Integrate LLM

In [6]:
# Install Transformers
!pip install transformers

from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')

def generate_insights(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = gpt_model.generate(inputs, max_length=150, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example insights generation
insights = generate_insights(query)
print("\nInsights:")
print(insights)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Insights:
What are the risk factors associated with Google and Tesla?

The risk factors for Tesla are:

The risk of a car accident.

The risk of a car accident in the United States.

The risk of a car accident in the United Kingdom.

The risk of a car accident in the United States.

The risk of a car accident in the United States.

The risk of a car accident in the United States.

The risk of a car accident in the United States.

The risk of a car accident in the United States.

The risk of a car accident in the United States.

The risk of a car accident in the United States.

The risk


Execution

In [7]:
!pip install streamlit pyngrok sentence-transformers faiss-cpu transformers PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [13]:
 %%writefile app.py
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import faiss
import numpy as np

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Function to create a FAISS index
def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index

# Function to query the FAISS index
def query_index(index, query_embedding, texts):
    D, I = index.search(query_embedding, k=5)
    return [texts[i] for i in I[0]]

# Function to get query embeddings
def get_query_embedding(query):
    return model.encode([query])

# Function to generate insights using GPT-2
def generate_insights(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = gpt_model.generate(inputs, max_length=150, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Load models
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
# Defining 'model' here, referencing the SentenceTransformer model
model = embedding_model

# Function to get query embeddings
def get_query_embedding(query):
    return model.encode([query])

# Load and process PDF files
alphabet_text = extract_text_from_pdf("Alphabet_10K.pdf")
tesla_text = extract_text_from_pdf("Tesla_10K.pdf")
uber_text = extract_text_from_pdf("Uber_10K.pdf")

alphabet_embeddings = embedding_model.encode(alphabet_text.split('\n'))
tesla_embeddings = embedding_model.encode(tesla_text.split('\n'))
uber_embeddings = embedding_model.encode(uber_text.split('\n'))

alphabet_index = create_faiss_index(alphabet_embeddings)
tesla_index = create_faiss_index(tesla_embeddings)
uber_index = create_faiss_index(uber_embeddings)

# Streamlit UI
st.title("Content Engine Chatbot")

query = st.text_input("Enter your query:")
if query:
    query_embedding = get_query_embedding(query)

    st.write("### Alphabet Inc.")
    alphabet_results = query_index(alphabet_index, query_embedding, alphabet_text.split('\n'))
    for result in alphabet_results:
        st.write(result)

    st.write("### Tesla, Inc.")
    tesla_results = query_index(tesla_index, query_embedding, tesla_text.split('\n'))
    for result in tesla_results:
        st.write(result)

    st.write("### Uber Technologies, Inc.")
    uber_results = query_index(uber_index, query_embedding, uber_text.split('\n'))
    for result in uber_results:
        st.write(result)

    st.write("### Insights")
    insights = generate_insights(query)
    st.write(insights)


Overwriting app.py


In [15]:
!pip install --upgrade pyngrok

from pyngrok import ngrok
ngrok.set_auth_token("2N13huDP2ANtbXSJQ7OQ2HvAShU_2pEtgBSQ4CM4B6mY6kTEF") # Removed extra space before this line

# Terminate all existing ngrok tunnels
ngrok.kill()

# Create a new ngrok tunnel, explicitly specifying HTTP protocol
public_url = ngrok.connect(8501, proto="http") # Specify protocol as "http"
print(f"Streamlit App URL: {public_url}")

Streamlit App URL: NgrokTunnel: "https://c45e-34-16-144-25.ngrok-free.app/" -> "http://localhost:8501"


In [16]:
import subprocess

# Run Streamlit app
subprocess.Popen(['streamlit', 'run', 'app.py'])




<Popen: returncode: None args: ['streamlit', 'run', 'app.py']>